In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime,date
import re
import json
from pandas.api.types import is_numeric_dtype

# from simple_salesforce import Salesforce, format_soql, format_external_id
# from simple_salesforce.exceptions import SalesforceMalformedRequest
# import simpler_sf
from sqlalchemy import create_engine
#import pymysql

# from mysql import connector


# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [2]:
# usuario = 'siempreread'
# contrasena = 'S13mpr3_Ch1l3'
# url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "MetLife_A"
plugin_autenticacion = "mysql_native_password"

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

#table.to_sql(name='rut_unico_tiempo',con=motor_mysql_mariadb,if_exists = 'append')


# archivo de salida modificar segun archivo
# archivo = 'C:/Users/jmartinez/Desktop/casos nuevos/clientes/1/casos_faltantes_segun_archivo.xlsx'

In [3]:
# table.to_sql(name='prueba',con=motor_mysql_mariadb,if_exists = 'append')
rut_unicos = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unicos WHERE LEFT(Rut_T, 1) <> '0' "
rut_unicos = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("-", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("–", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(" ", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.upper()
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.strip()

rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("-", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("–", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(" ", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.upper()
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.strip()

rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(",", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(".", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("-", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┐", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("═", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("╔", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┌", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.title()
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.strip()

rut_unicos.shape

""" rut_unicos.to_excel(
    "F:/Empresas SF/reportes/rut_unicos_hasta_Marzo_2024.xlsx", index=False
) """

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\386836739.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\386836739.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\386836739.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Nombre"] = rut_unicos["Nombre"].str.r

' rut_unicos.to_excel(\n    "F:/Empresas SF/reportes/rut_unicos_hasta_Marzo_2024.xlsx", index=False\n) '

In [4]:
rut_unico_tiempo_ = pd.DataFrame()
sql = "SELECT Llave FROM MetLife_A.rut_unico_tiempo "
rut_unico_tiempo_ = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unico_tiempo_.shape

(7277009, 1)

In [5]:
rut_unico_tiempo_.head(9)

,Llave
0,34002302717805539912023
1,34000243618617790812023
2,34002273710579623412023
3,34002317316849726112023
4,34000428510812004512023
5,3400230029816165112023
6,34002275520984896112023
7,34001998912596172K12023
8,34001207115907946512023


In [6]:
def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        #print("date_string =", date_string)
        #print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        #print("date_object =", date_object)
        #print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

In [7]:
mes = 8
año = "2023"
fecha = año + "-" + str(mes) + "-" + "01"  
mesesDic = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

str(mesesDic[mes])



sftp = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/sftp.xlsx"


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"


derty = pd.read_excel(sftp)

In [8]:
derty.shape

(160371, 20)

In [9]:
metlife_sftp = pd.DataFrame()
metlife_sftp = derty.copy()

In [10]:


metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("-", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("–", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(" ", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.upper()
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.strip()

metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("-", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("–", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(" ", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.upper()
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.strip()

metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(",", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(".", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("-", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┐", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("═", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("╔", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┌", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.title()
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.strip()

metlife_sftp["poliza"] = metlife_sftp["poliza"].astype(str)
metlife_sftp["poliza"] = metlife_sftp["poliza"].str.strip()



#*******************sacar primer cero
metlife_sftp.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\3662042864.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\3662042864.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\3662042864.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["Nombre"] = metlife_sftp["Nomb

(160371, 20)

In [11]:
metlife_sftp.head()

,poliza,RUTASEGURADO,DV,rut1,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut2,fdgd,FNAC,SEXO,DIRECCION,COMUNA,CIUDAD,TELEFONO,RELACION,desde,hasta,Nombre,EMAIL,NROCELULAR
0,340006458,12345,5,123455,12345,5,123455,1,29014,M,NaN,NaN,NaN,NaN,AS,2016-03-01,NaT,Puebla Abarca Nelson Hernan,NaN,NaN
1,340011238,1701221,5,17012215,1701221,5,17012215,1,12789,F,MONTENEGRO 189 DPTO103-B,ÑUÑOA,SANTIAGO,229811223,AS,2016-08-01,NaT,Castro Grimberg Ester,ester.castro1@gmail.com,NaN
2,340011238,2239787,7,22397877,2239787,7,22397877,1,10443,M,CONCEPCION 945 PB PATRONA DE CHILE,MAIPÚ,NaN,NaN,AS,2016-08-01,NaT,Salazar Fernando,NaN,NaN
3,340011238,2266265,1,22662651,2266265,1,22662651,1,9339,F,NaN,NaN,NaN,NaN,AS,2016-08-01,NaT,Bonvallet Lohaus Berta,NaN,NaN
4,340011238,2626779,K,2626779K,2626779,K,2626779K,1,9663,M,A ESPINOZA 18 POBL M MONTT,PUERTO MONTT,NaN,65-255795 / 68717350,AS,2016-08-01,NaT,Oliva Rosas Hector,NaN,NaN


In [12]:
def siPrimerCero(y):    
    if len(y) <= 4:
        y =  "NONE"
    if y[0] == "0":
        y = y[1:]
    return str(y)
    

In [13]:
metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp = metlife_sftp[(metlife_sftp["rut1"] != "NONE")]
metlife_sftp = metlife_sftp[(metlife_sftp["rut2"] != "NONE")]


metlife_sftp["Titular"] = np.where(metlife_sftp["rut1"] == metlife_sftp["rut2"],1,2 )
metlife_sftp = metlife_sftp[(metlife_sftp["Titular"] == 1)]
metlife_sftp = metlife_sftp.drop_duplicates(
    [
        "rut1","poliza",
    ],
    keep="last",
)

metlife_sftp.shape

(160371, 21)

In [14]:
correo = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/correo.xlsx"
dertyc = pd.read_excel(correo)
metlife_correo = dertyc.copy()

# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("-", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("–", "")
metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(" ", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.upper()
metlife_correo["rut1"] = metlife_correo["rut1"].str.strip()

metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("-", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("–", "")
metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(" ", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.upper()
metlife_correo["rut2"] = metlife_correo["rut2"].str.strip()

metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(",", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(".", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("-", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┐", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("═", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("╔", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┌", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.title()
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.strip()

metlife_correo["poliza"] = metlife_correo["poliza"].astype(str)
metlife_correo["poliza"] = metlife_correo["poliza"].str.strip()




#*******************sacar primer cero
metlife_correo.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\2005110109.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\2005110109.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\2005110109.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["Nombre"] = metlife

(160371, 20)

In [15]:
metlife_correo["rut1"] = metlife_correo["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo = metlife_correo[(metlife_correo["rut1"] != "NONE")]
metlife_correo = metlife_correo[(metlife_correo["rut2"] != "NONE")]

metlife_correo["Titular"] = np.where(metlife_correo["rut1"] == metlife_correo["rut2"],1,2 )
metlife_correo = metlife_correo[(metlife_correo["Titular"] == 1)]
metlife_correo = metlife_correo.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)

metlife_correo.shape

(160371, 21)

In [16]:
web = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/web.xlsx"
dertyd = pd.read_excel(web)


In [17]:
dertyd.head(9)

,poliza,RUTASEGURADO,DV,rut1,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut2,fdgd,FNAC,SEXO,DIRECCION,COMUNA,CIUDAD,TELEFONO,RELACION,desde,hasta,Nombre,EMAIL,NROCELULAR
0,340006458,12345,5,123455,12345,5,123455,1,29014,M,NaN,NaN,NaN,NaN,AS,2016-03-01,NaT,Puebla Abarca Nelson Hernan,NaN,NaN
1,340011238,1701221,5,17012215,1701221,5,17012215,1,12789,F,MONTENEGRO 189 DPTO103-B,ÑUÑOA,SANTIAGO,229811223,AS,2016-08-01,NaT,Castro Grimberg Ester,ester.castro1@gmail.com,NaN
2,340011238,2239787,7,22397877,2239787,7,22397877,1,10443,M,CONCEPCION 945 PB PATRONA DE CHILE,MAIPÚ,NaN,NaN,AS,2016-08-01,NaT,Salazar Fernando,NaN,NaN
3,340011238,2266265,1,22662651,2266265,1,22662651,1,9339,F,NaN,NaN,NaN,NaN,AS,2016-08-01,NaT,Bonvallet Lohaus Berta,NaN,NaN
4,340011238,2626779,K,2626779K,2626779,K,2626779K,1,9663,M,A ESPINOZA 18 POBL M MONTT,PUERTO MONTT,NaN,65-255795 / 68717350,AS,2016-08-01,NaT,Oliva Rosas Hector,NaN,NaN
5,340022643,2693013,8,26930138,2693013,8,26930138,1,10612,M,ORTUZAR 1064,ÑUÑOA,NaN,NaN,AS,2022-09-01,NaT,Guzman Campos Mario,NaN,NaN
6,340011238,2710995,0,27109950,2710995,0,27109950,1,11982,F,PTO NATALES 6415 V BRASIL,LA GRANJA,NaN,225258332,AS,2016-08-01,NaT,Cardenas Elgueta Virginia,notiene@metlife.cl,NaN
7,340011238,2747250,8,27472508,2747250,8,27472508,1,11862,M,GOTEMBURGO 220 DPTO 191,LAS CONDES,NaN,2110783,AS,2016-08-01,NaT,Rodriguez Lacunza Alfonso,eliana.rodriguez@esmax.cl,NaN
8,340019776,2884455,7,28844557,2884455,7,28844557,1,13701,M,CURAMAVIDA 678,LAS CONDES,NaN,NaN,AS,2022-06-01,NaT,Von Appen Behrmann Wolf Hansalbe,WVONAPPEN@ULTRAMAR.CL,NaN


In [18]:
metlife_web = dertyd.copy()

In [19]:
metlife_web.head()

,poliza,RUTASEGURADO,DV,rut1,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut2,fdgd,FNAC,SEXO,DIRECCION,COMUNA,CIUDAD,TELEFONO,RELACION,desde,hasta,Nombre,EMAIL,NROCELULAR
0,340006458,12345,5,123455,12345,5,123455,1,29014,M,NaN,NaN,NaN,NaN,AS,2016-03-01,NaT,Puebla Abarca Nelson Hernan,NaN,NaN
1,340011238,1701221,5,17012215,1701221,5,17012215,1,12789,F,MONTENEGRO 189 DPTO103-B,ÑUÑOA,SANTIAGO,229811223,AS,2016-08-01,NaT,Castro Grimberg Ester,ester.castro1@gmail.com,NaN
2,340011238,2239787,7,22397877,2239787,7,22397877,1,10443,M,CONCEPCION 945 PB PATRONA DE CHILE,MAIPÚ,NaN,NaN,AS,2016-08-01,NaT,Salazar Fernando,NaN,NaN
3,340011238,2266265,1,22662651,2266265,1,22662651,1,9339,F,NaN,NaN,NaN,NaN,AS,2016-08-01,NaT,Bonvallet Lohaus Berta,NaN,NaN
4,340011238,2626779,K,2626779K,2626779,K,2626779K,1,9663,M,A ESPINOZA 18 POBL M MONTT,PUERTO MONTT,NaN,65-255795 / 68717350,AS,2016-08-01,NaT,Oliva Rosas Hector,NaN,NaN


In [20]:


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_web["rut1"] = metlife_web["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut1"] = metlife_web["rut1"].str.replace("-", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace("–", "")
metlife_web["rut1"] = metlife_web["rut1"].astype(str)
metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace(" ", "")
metlife_web["rut1"] = metlife_web["rut1"].str.upper()
metlife_web["rut1"] = metlife_web["rut1"].str.strip()

metlife_web["rut2"] = metlife_web["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].str.replace("-", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace("–", "")
metlife_web["rut2"] = metlife_web["rut2"].astype(str)
metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace(" ", "")
metlife_web["rut2"] = metlife_web["rut2"].str.upper()
metlife_web["rut2"] = metlife_web["rut2"].str.strip()

metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(",", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(".", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("-", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┐", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("═", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("╔", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┌", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.title()
metlife_web["Nombre"] = metlife_web["Nombre"].str.strip()

metlife_web["poliza"] = metlife_web["poliza"].astype(str)
metlife_web["poliza"] = metlife_web["poliza"].str.strip()

metlife_web["rut1"] = metlife_web["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web = metlife_web[(metlife_web["rut1"] != "NONE")]
metlife_web = metlife_web[(metlife_web["rut2"] != "NONE")]
metlife_web.shape

metlife_web["Titular"] = np.where(metlife_web["rut1"] == metlife_web["rut2"],1,2 )
metlife_web = metlife_web[(metlife_web["Titular"] == 1)]

metlife_web = metlife_web.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)


#*******************sacar primer cero
metlife_web.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\2932449918.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\2932449918.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16192\2932449918.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["Nombre"] = metlife_web["Nombre"].

(160371, 21)

In [21]:
subir_correo = pd.DataFrame()
subir_correo["poliza_rut"] = metlife_correo["poliza"]
subir_correo["rut_rut"] = metlife_correo["rut1"]
subir_correo["nombre_rut"] = metlife_correo["Nombre"]
subir_correo["mes_rut"] = str(mes)
subir_correo["año_rut"] = str(año)
#subir_correo["fecha_rut"] = metlife_correo["fecha"]
subir_correo["Llave"] = metlife_correo["poliza"] + metlife_correo["rut1"] + str(mes) + str(año)
subir_correo["desde_rut"] = metlife_correo["desde"].apply(lambda y: fechaValida(y))
subir_correo["hasta_rut"] = metlife_correo["hasta"].apply(lambda y: fechaValida(y))

subir_sql = pd.DataFrame()
subir_sql["poliza_rut"] = metlife_sftp["poliza"]
subir_sql["rut_rut"] = metlife_sftp["rut1"]
subir_sql["nombre_rut"] = metlife_sftp["Nombre"]
subir_sql["mes_rut"] = str(mes)
subir_sql["año_rut"] = str(año)
#subir_sql["fecha_rut"] = metlife_sftp["fecha"]
subir_sql["Llave"] = metlife_sftp["poliza"] + metlife_sftp["rut1"] + str(mes) + str(año)
subir_sql["desde_rut"] = metlife_sftp["desde"].apply(lambda y: fechaValida(y))
subir_sql["hasta_rut"] = metlife_sftp["hasta"].apply(lambda y: fechaValida(y)) 

subir_web = pd.DataFrame()
subir_web["poliza_rut"] = metlife_web["poliza"]
subir_web["rut_rut"] = metlife_web["rut1"]
subir_web["nombre_rut"] = metlife_web["Nombre"]
subir_web["mes_rut"] = str(mes)
subir_web["año_rut"] = str(año)
#subir_web["fecha_rut"] = metlife_web["fecha"]
subir_web["Llave"] = metlife_web["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_web["desde_rut"] = metlife_web["desde"].apply(lambda y: fechaValida(y))
subir_web["hasta_rut"] = metlife_web["hasta"].apply(lambda y: fechaValida(y))

mes_metlife = pd.DataFrame()
mes_metlife = pd.concat(
    [
        subir_web,
        subir_sql,
        subir_correo,
        
    ],
    axis=0,
)
mes_metlife["fecha_rut"] = pd.to_datetime(fecha, yearfirst=True)

mes_metlife = mes_metlife.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)
mes_metlife.head()

,poliza_rut,rut_rut,nombre_rut,mes_rut,año_rut,Llave,desde_rut,hasta_rut,fecha_rut
0,340006458,123455,Puebla Abarca Nelson Hernan,8,2023,34000645812345582023,2016-03-01,2024-03-19 13:12:49,2023-08-01
1,340011238,17012215,Castro Grimberg Ester,8,2023,3400112381701221582023,2016-08-01,2024-03-19 13:12:49,2023-08-01
2,340011238,22397877,Salazar Fernando,8,2023,3400112382239787782023,2016-08-01,2024-03-19 13:12:49,2023-08-01
3,340011238,22662651,Bonvallet Lohaus Berta,8,2023,3400112382266265182023,2016-08-01,2024-03-19 13:12:49,2023-08-01
4,340011238,2626779K,Oliva Rosas Hector,8,2023,3400112382626779K82023,2016-08-01,2024-03-19 13:12:49,2023-08-01


//////////////////////////////////////////////////////////////////////////

NUMERO RUTS POR POLIZA
****************************************************************

In [22]:
mes_metlife.shape
print ( str(mes) + "-" + str(año) + " " + str(mes_metlife.shape[0]))

8-2023 160371


////////////////////////////////////////////////

NUMERO RUT UNICOS
------------------------------------------------------------------------------------------------

In [23]:
mes_metlife2 = mes_metlife.drop_duplicates(
    [
        "rut_rut"
    ],
    keep="last",
)
mes_metlife2.shape
print (str(mes) + "-" + str(año) + " " + str(mes_metlife2.shape[0]))

8-2023 160371


In [24]:
#subir_web["Llave"]
mes_metlife["esta"] =  mes_metlife["Llave"].isin(rut_unico_tiempo_["Llave"])
mes_metlife = mes_metlife[(mes_metlife["esta"] == False)]

mes_metlife = mes_metlife[
    [
        "poliza_rut",
        "rut_rut",
        "nombre_rut",
        "mes_rut",
        "año_rut",
        "Llave",
        "desde_rut",
        "hasta_rut",
    ]
]
mes_metlife.shape

(0, 8)

In [25]:
mes_metlife.to_sql(
    name="rut_unico_tiempo",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

0

In [26]:
subir_unicos = pd.DataFrame()
subir_unicos["Rut_T"] = mes_metlife2["rut_rut"]
subir_unicos["Rut_C"] = mes_metlife2["rut_rut"]
subir_unicos["Nombre"] = mes_metlife2["nombre_rut"]
subir_unicos["Llave"] = mes_metlife2["rut_rut"] +  mes_metlife2["rut_rut"]


subir_unicos.shape

(160371, 4)

In [27]:
subir_unicos["esta"] =  subir_unicos["Rut_T"].isin(rut_unicos["Rut_T"])
subir_unicos = subir_unicos[(subir_unicos["esta"] == False)]

subir_unicos = subir_unicos[
    [
        "Rut_T",
        "Rut_C",
        "Nombre",
        "Llave",
    ]
]
subir_unicos.shape
subir_unicos.to_sql(
    name="rut_unicos",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

0

In [28]:



rut_unicos["esta"] =  rut_unicos["Rut_T"].isin(mes_metlife2["rut_rut"])
rut_unicos = rut_unicos[(rut_unicos["esta"] == False)]


In [29]:
rut_unicos.shape

(323895, 6)

In [30]:
rut_unicos.to_excel(
    "F:/Empresas SF/reportes/rut_unicos_No_Estan_SFTP_Agosto_2023.xlsx", index=False
)

In [31]:
sftp = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/poliza.xlsx"
poliza = pd.read_excel(sftp)

In [32]:
poliza.head()

,poliza,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,340023542,Uber Rides Chile SPA,Uber Rides Chile SPA,-,MetLife,Uber,MetLife Orienta,Chile,True,a3yRN0000006sFLYAY,340023542,1
1,340023542,Uber Rides Chile SPA,Uber Rides Chile SPA,-,MetLife,Uber,MetLife Orienta,Chile,True,a3yRN0000006sFMYAY,340023542,1
2,340019779,Ultramar Agencia Maritima Ltda.,Ultramar Agencia Maritima Ltda.,-,MetLife,Ultramar - Agencias,MetLife Orienta,Chile,True,a3yRN0000006sFNYAY,340019779,1
3,340017378,Ultrapoly S.A.,Ultrapoly S.A.,-,MetLife,CTTA Codelco - El Teniente,MetLife Orienta,Chile,False,a3yRN0000006sFOYAY,340017378,1
4,340023511,Ultrapoly S.A.,Ultrapoly S.A.,-,MetLife,Agema EECC - DCH,MetLife Orienta,Chile,False,a3yRN0000006sFPYAY,340023511,1


In [33]:
rut_unico_tiempo_ = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unico_tiempo "
rut_unico_tiempo_ = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unico_tiempo_.shape

(7277009, 10)

In [34]:
rut_unicos = pd.merge(
    left=rut_unicos,
    right=rut_unico_tiempo_,
    how="left",
    left_on="Rut_T",
    right_on="rut_rut",
)

#rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [35]:
rut_unicos.shape

(3842268, 16)

In [36]:
rut_unicos.head()

,Id_x,Rut_T,Rut_C,Nombre,Llave_x,esta,Id_y,poliza_rut,rut_rut,nombre_rut,mes_rut,año_rut,fecha_rut,Llave_y,desde_rut,hasta_rut
0,1,209848961,209848961,Martina Isidora Sanhueza Almendra,209848961209848961,False,7,340022755,209848961,Martina Isidora Sanhueza Almendra,1,2023,None,34002275520984896112023,1970-01-01,1970-01-01
1,2,159079465,159079465,Jose Gonzalez Espinoza,159079465159079465,False,9,340012071,159079465,Jose Gonzalez Espinoza,1,2023,None,34001207115907946512023,1970-01-01,1970-01-01
2,2,159079465,159079465,Jose Gonzalez Espinoza,159079465159079465,False,3535277,340012071,159079465,Jose Gonzalez Espinoza,9,2023,None,34001207115907946592023,2024-03-15,2024-03-15
3,2,159079465,159079465,Jose Gonzalez Espinoza,159079465159079465,False,4942957,340012071,159079465,Gonzalez Espinoza Jose Patricio,2,2024,None,34001207115907946522024,2024-03-15,2024-03-15
4,2,159079465,159079465,Jose Gonzalez Espinoza,159079465159079465,False,7122428,340012071,159079465,Gonzalez Espinoza Jose Patricio,1,2028,None,34001207115907946512028,2022-11-01,2024-03-18


In [37]:
rut_unicos = rut_unicos.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)
rut_unicos.shape

(427905, 16)

In [38]:
rut_unicos = pd.merge(
    left=rut_unicos,
    right=poliza,
    how="left",
    left_on="poliza_rut",
    right_on="poliza",
)

In [39]:
rut_unicos.shape

(434505, 28)

In [40]:
rut_unicos.head()


,Id_x,Rut_T,Rut_C,Nombre,Llave_x,esta,Id_y,poliza_rut,rut_rut,nombre_rut,...,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,1,209848961,209848961,Martina Isidora Sanhueza Almendra,209848961209848961,False,7,340022755,209848961,Martina Isidora Sanhueza Almendra,...,Walmart,-,MetLife,Walmart,MetLife Orienta,Chile,True,a3yRN0000006tkaYAA,340022755,1.0
1,2,159079465,159079465,Jose Gonzalez Espinoza,159079465159079465,False,7122428,340012071,159079465,Gonzalez Espinoza Jose Patricio,...,Brinks SPA,-,MetLife,Brinks,MetLife Orienta,Chile,True,a3yRN0000006t9GYAQ,340012071,1.0
2,3,156070432,156070432,Pablo Cesar Aguirre Rodriguez,156070432156070432,False,11,340022755,156070432,Pablo Cesar Aguirre Rodriguez,...,Walmart,-,MetLife,Walmart,MetLife Orienta,Chile,True,a3yRN0000006tkaYAA,340022755,1.0
3,5,155558571,155558571,Danilo Enrique Santis Barrera,155558571155558571,False,19,340003406,155558571,Danilo Enrique Santis Barrera,...,TB Tobaco,-,MetLife,TB Tobaco,MetLife Orienta,Chile,True,a3yRN0000006tnkYAA,340003406,1.0
4,6,193277365,193277365,Alejandro Jesus Alvarez Pacheco,193277365193277365,False,20,340003406,193277365,Alejandro Jesus Alvarez Pacheco,...,TB Tobaco,-,MetLife,TB Tobaco,MetLife Orienta,Chile,True,a3yRN0000006tnkYAA,340003406,1.0


In [41]:
rut_unicos.to_excel(
    "F:/Empresas SF/reportes/rut_unicos_No_Estan_SFTP_Agosto_2023_2.xlsx", index=False
)